# Frozen Molecules and redefine a center to move/rotate Molecules:
**Task:** 
1. Choose any Molecule to be FROZEN (fixed) and it can NOT be moved/rotated
2. Redefine a different 3D point as a center for our cluster

## Iodine and water pentamer [$I$($H_{2}O$)$_6$]:
Cartesian coordinates [Angstrom]:

    I    0.00000000	     0.00000000	     0.00000000
    O    0.00000000	     0.00000000	     0.00000000
    H    0.58708000	     0.75754000	     0.00000000
    H   -0.58708000	     0.75754000	     0.00000000
    O    0.00000000	     0.00000000	     0.00000000
    H    0.58708000	     0.75754000	     0.00000000
    H   -0.58708000	     0.75754000	     0.00000000
    O    0.00000000	     0.00000000	     0.00000000
    H    0.58708000	     0.75754000	     0.00000000
    H   -0.58708000	     0.75754000	     0.00000000
    O    0.00000000	     0.00000000	     0.00000000
    H    0.58708000	     0.75754000	     0.00000000
    H   -0.58708000	     0.75754000	     0.00000000
    O    0.00000000	     0.00000000	     0.00000000
    H    0.58708000	     0.75754000	     0.00000000
    H   -0.58708000	     0.75754000	     0.00000000

> **_NOTE:_**   
> `translate` and `rotate` methods return a brand new Molecule object

In [1]:
# crating the path (PYTHONPATH) to our module.
# assuming that our 'src' directory is out ('..') of our current directory 
import os
import sys
module_path = os.path.abspath(os.path.join('..'))

if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
# importing de Molecule Class
from src.base_molecule import Molecule, Cluster

In [3]:
# let's create a cluster with five (5) water molecules and one atom (Iodine)
water_molecule=[("O", 0, 0, 0), ("H", 0.58708, 0.75754, 0), ("H", -0.58708, 0.75754, 0)]
iodine =[("I", 0, 0, 0)]

# if we want an accurate orden in our cluster, we should use COMMA 
# for example, Iodine as the first molecule in our cluster
w5_I = Cluster(Molecule(iodine), 5 * Cluster(water_molecule))


# ALWAYS, you have SIX (6) individual molecules/atoms in your 'w5_I' cluster
print(f"Number of individual molecules/atoms: {w5_I.total_molecules}")
print(w5_I.xyz)


Number of individual molecules/atoms: 6
	16
-- charge= 0 and multiplicity= 1 --
I     	     0.00000000	     0.00000000	     0.00000000
O     	     0.00000000	     0.00000000	     0.00000000
H     	     0.58708000	     0.75754000	     0.00000000
H     	    -0.58708000	     0.75754000	     0.00000000
O     	     0.00000000	     0.00000000	     0.00000000
H     	     0.58708000	     0.75754000	     0.00000000
H     	    -0.58708000	     0.75754000	     0.00000000
O     	     0.00000000	     0.00000000	     0.00000000
H     	     0.58708000	     0.75754000	     0.00000000
H     	    -0.58708000	     0.75754000	     0.00000000
O     	     0.00000000	     0.00000000	     0.00000000
H     	     0.58708000	     0.75754000	     0.00000000
H     	    -0.58708000	     0.75754000	     0.00000000
O     	     0.00000000	     0.00000000	     0.00000000
H     	     0.58708000	     0.75754000	     0.00000000
H     	    -0.58708000	     0.75754000	     0.00000000



In [4]:
from copy import deepcopy


# creating a brand new copy when 'I' will NOT be moved or rotated 
w5 = deepcopy(w5_I)

# define the size for our sphere (boundary conditions)
sphere_radius = 10

# setting a sphere radius for the Cluster spherical boundary conditions
w5.sphere_radius = sphere_radius

# let's redefine a new center for our sphere
new_x, new_y, new_z = 20, 20, 20
w5.sphere_center = (new_x, new_y, new_z)

# let's add another atoms (any) to show the new sphere center
w5 = w5.add_molecule([("F", new_x, new_y, new_z)])

# frozen molecule to avoid be moved or rotated
w5.frozen_molecule = [0, 6]

print(w5)
print(w5.xyz)

Cluster of 7 moleculesand 17 total atoms
molecule 0 (with 1 atoms):
     -> atoms: [('I', 0, 0, 0)]
     -> charge: 0
     -> multiplicity: 1
molecule 1 (with 3 atoms):
     -> atoms: [('O', 0, 0, 0), ('H', 0.58708, 0.75754, 0), ('H', -0.58708, 0.75754, 0)]
     -> charge: 0
     -> multiplicity: 1
molecule 2 (with 3 atoms):
     -> atoms: [('O', 0, 0, 0), ('H', 0.58708, 0.75754, 0), ('H', -0.58708, 0.75754, 0)]
     -> charge: 0
     -> multiplicity: 1
molecule 3 (with 3 atoms):
     -> atoms: [('O', 0, 0, 0), ('H', 0.58708, 0.75754, 0), ('H', -0.58708, 0.75754, 0)]
     -> charge: 0
     -> multiplicity: 1
molecule 4 (with 3 atoms):
     -> atoms: [('O', 0, 0, 0), ('H', 0.58708, 0.75754, 0), ('H', -0.58708, 0.75754, 0)]
     -> charge: 0
     -> multiplicity: 1
molecule 5 (with 3 atoms):
     -> atoms: [('O', 0, 0, 0), ('H', 0.58708, 0.75754, 0), ('H', -0.58708, 0.75754, 0)]
     -> charge: 0
     -> multiplicity: 1
molecule 6 (with 1 atoms):
     -> atoms: [('F', 20, 20, 20)]
     -

In [5]:
from numpy import random
random_gen = random.default_rng(1234)

# to save snapshot and show as a movie
w5_xyz = ""

total_steps = 10000

for i in range(total_steps):
    # molecule to be selected between [0, w.total_molecules]
    # frozen molecules will not be moved or rotated
    mol = random_gen.choice(w5.total_molecules)

    # angle between [0, 360)
    ax = random_gen.uniform() * 360
    ay = random_gen.uniform() * 360
    az = random_gen.uniform() * 360

    # moving between [-step, +step]
    step = sphere_radius / 2
    tx = step * (random_gen.uniform() - 0.5)
    ty = step * (random_gen.uniform() - 0.5)
    tz = step * (random_gen.uniform() - 0.5)

    # saving coordinates as a string
    # w5_xyz += w5.xyz
    if i % 100 == 0: 
        w5_xyz += w5.xyz

    w5 = w5.translate(mol, x=tx, y=ty, z=tz).rotate(
        mol, x=ax, y=ay, z=az
    )
    # printing current step
    print(
        f"\r progress {100.0*((i + 1)/total_steps):.2f}"
        f" % -- step {i + 1}/{total_steps}", end=''
        )

# -------------------------------------------------------
print("\n *** JOB DONE ***")
print(f"after {total_steps} steps\n")
print(w5.xyz)

 progress 100.00 % -- step 10000/10000
 *** JOB DONE ***
after 10000 steps

	17
-- charge= 0 and multiplicity= 1 --
I     	     0.00000000	     0.00000000	     0.00000000
O     	    15.96555585	    12.42177298	    21.68757610
H     	    16.31042232	    12.86075918	    20.90854593
H     	    15.72961259	    11.84171574	    20.96204584
O     	    28.46434945	    21.19527486	    18.35483789
H     	    28.40442417	    20.99447145	    17.41962843
H     	    28.35461553	    22.04905197	    17.93346764
O     	    19.10666071	    27.74846904	    16.00219683
H     	    19.23574495	    28.69767128	    16.03189770
H     	    19.70648876	    27.88537360	    16.73703914
O     	    19.90738931	    28.85223860	    24.82614316
H     	    19.53729296	    28.58178388	    23.98447059
H     	    20.27207774	    27.97908619	    24.67403460
O     	    13.76413345	    20.66241925	    27.48851229
H     	    13.80268044	    21.21247912	    28.27239880
H     	    13.53303732	    20.07158039	    28.20686721
F   

!pip install py3Dmol
import py3Dmol

In [6]:
# py3Dmol: a simple IPython/Jupyter widget to embed an interactive 3Dmol.js viewer in a notebook.
!pip install py3Dmol
import py3Dmol

In [7]:
# 20% to check visually that everything is inside
sr = w5.sphere_radius * 1.2

# sphere center
cx, cy, cz = w5.sphere_center
sc = {"x": cx, "y": cy, "z": cz}

# axes
x_axis = {'start': {'x': cx-sr, 'y':cy, 'z':cz}, 'end': {'x': cx+sr, 'y':cy, 'z':cz}}
y_axis = {'start': {'x': cx, 'y':cy-sr, 'z':cz}, 'end': {'x': cx, 'y':cy+sr, 'z':cz}}
z_axis = {'start': {'x': cx, 'y':cy, 'z':cz-sr}, 'end': {'x': cx, 'y':cy, 'z':cz+sr}}

# starting visualization
xyz_view = py3Dmol.view(width=700, height=500)#, linked=False, viewergrid=(2,2))
xyz_view.addModelsAsFrames(w5_xyz,'xyz')
# xyz_view.setStyle({'stick': {}})
xyz_view.setStyle({'sphere': {'radius': 0.8}})
xyz_view.addSphere({'center': sc, 
        'radius': sr, 
        'color' :'yellow',
        'alpha': 0.5,
        })

# animation base on several XYZ coordinates snapshots
xyz_view.animate({'loop': "forward", 'speed': 1, 'reps': 1})

# cartesian 3D axes
xyz_view.addLine(x_axis)
xyz_view.addLine(y_axis)
xyz_view.addLine(z_axis)

xyz_view.addLabel("x", {
        'position':x_axis["end"],
        'inFront':'true',
        'fontSize':20,
        'showBackground':'false',
        'fontColor': 'black',
        })
xyz_view.addLabel("y", {
        'position':y_axis["end"],
        'inFront':'true',
        'fontSize':20,
        'showBackground':'false',
        'fontColor': 'black',
        })
xyz_view.addLabel("z", {
        'position':z_axis["end"],
        'inFront':'true',
        'fontSize':20,
        'showBackground':'false',
        'fontColor': 'black',
        })

xyz_view.zoomTo()
xyz_view.show()


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

# A better example...

In [8]:
from data.molecules_coordinates import metal_complex 

complex = Cluster(metal_complex, 10 * Cluster(water_molecule))